Il file 'Data.csv' contiene i dati relativi al consumo di energia elettrica della macchina del caffè di Alexide.
La colonna Date rappresenta data e ora della registrazione del sensore. Si noti che le date sono espresse in UTC. Per convertirle nella timezone locale utilizzare
df['Date'] = pd.to_datetime(df['Date'], utc = True).dt.tz_convert('Europe/Berlin')
La colonna Energy rappresenta il consumo di energia totale, espresso in Wmin (Watt minuto), a partire dall'accensione del sensore. Si noti che il sensore è stato riavviato diverse volte nel periodo di osservazione e, a seguito di ciascun riavvio, il conteggio è ripartito da 0.
La colonna *Topic** rappresenta il topic del messaggio mqtt ricevuto dal sensore.

Rappresentare i consumi totali in kWh (kiloWatt ora) con una heatmap che abbia sull'asse x le ore del giorno e sull'asse y il giorno della settimana.

importo le librerie con le quali abbiamo lavorato

In [ ]:
import numpy as np
import pandas as pd
import os
import glob2
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

carico il file presente nella cartella memorizzandolo nel nostro Data Frame

In [ ]:
df = pd.read_csv('data2.csv')

visualizzo il DF che ho caricato

In [ ]:
df

visualizzo le relative informazioni su colonne tipi di dati ed eventuali valori nulli

In [ ]:
df.info()

imposto la data nella colonna Date, la allineo al fuso orario

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], utc = True).dt.tz_convert('Europe/Berlin')

controllo l'impostazione della data che sia cambiata

In [ ]:
df.info()

ordino le colonne per data in ordine crescente

In [ ]:
df = df.sort_values('Date')

creo una colonna nella quale calocolo la differenza di tempo fra un messagio MQTT e quello successivo al fine di calcolare in seguito il consumo

In [ ]:
df['DelTim'] = (df['Date'].shift(-1) - df['Date'])
df.info()

considero solo i messaggi MQTT inferiori ai 30 minuti come distanza di arrivo

In [ ]:
df = df[df['DelTim'] < pd.Timedelta(30, 'm')]

trasformo la colonna dei Delta dei minuti in ore

In [ ]:
df['DelTim'] = df['DelTim'].dt.total_seconds()/60

scarto gli eventuali valori 'Nulli'

In [ ]:
df = df.dropna()

visualizzo le informazioni dei DF e controllo se tutti i campi sono corretti e se non ci sono
più valori nulli

In [ ]:
df.info()

visualizzo il DF completo di valori

In [ ]:
df

converto l'energia impiegata in KWa (chilowatt)

In [ ]:
df['Energy'] = df['Energy']/1000

creo la colonna del consumo orario (KW/h)

In [ ]:
df['ConsHour'] = df['DelTim']*df['Energy']

visualizzo il DF completo di valori e con le nuove colonne inserite

In [ ]:
df

visualizzo le informazioni sulle colonne nuove del DF

In [ ]:
df.info()

eseguo il Resample sulla base delle ore, aggrego le colonne di data sommando la colonna del 
consumo, visualizzo solo le colonne di data 'Date' e consumo 'ConsHour'

In [ ]:
df = df.resample('H', on='Date').agg({'ConsHour': 'sum'}).reset_index()

visualizzo il DF completo di valori e con le nuove modifiche fatte, senza le colonne

In [ ]:
df

estrapolo dalla colonna 'Date' il nome del giorno col metodo (dayofweek)

In [ ]:
df['DoWeek'] = df['Date'].dt.dayofweek

estrapolo dalla colonna 'Date' l'ora  del giorno col metodo (hour)

In [ ]:
df['Hour'] = df['Date'].dt.hour

a questo punto scarto la colonna 'Date' e la cancello

In [ ]:
df = df.drop(columns=['Date'])

formatto il mio DF in modo da raggruppare per giorno della settimana e ora, sommando i dati 
appunto in queste colonne ottenendo il consumo di quel giorno della settimana e per quel ora

In [ ]:
df = df.groupby(['DoWeek', 'Hour']).sum()

visualizzo il risultato nel DF

In [ ]:
df

con la funzione log (logaritmo) si riducono i picchi di valori applicata ai consumi

In [ ]:
df['ConsHour'] = np.log(df['ConsHour'])

normalizzo la colonna dei consumi portandola a valori compresi fra 0 e 1 per una 
rappresentazione migliore

In [ ]:
df['ConsHour'] = (df['ConsHour'] - df['ConsHour'].min())/(df['ConsHour'].max() - df['ConsHour'].min())

In [ ]:
df

col metodo unstack creo per ogni ora del giorno la sua relativa colonna

In [ ]:
df = df.unstack()

scarto la riga del titolo 'ConsHour' del mio DF mantenendo solo le altre righe

In [ ]:
df.columns = df.columns.droplevel(0)

visualizzo di nuovo il mio DF prima della  Hitmap

In [ ]:
df

con Seaborn stampo la hitmap considerando sul asse x le ore del giorno e sul asse y il giorno 
della settimana, in colorazione crescente per ogni incrocio nel grafico ho il relativo consumo

In [ ]:
sns.heatmap(df, linewidths=.5)